In [3]:
# %matplotlib inline
# %matplotlib notebook
import tqdm, copy
import random, os
import pandas as pd
from tqdm import trange
import matplotlib.pyplot as plt
from collections import defaultdict
from overcooked_ai_py.utils import save_pickle
from human_aware_rl.utils import set_global_seed
from human_aware_rl.human.process_dataframes import *
# from human_aware_rl.static import *
import pickle as pkl

set_global_seed(1884)
import ast
import json

from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import os
from ast import literal_eval
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


# Define Helper Functions to Open Data

In [4]:
def import_cleaned_data(filename):
    

    with open(filename,'rb') as file:
        data = pkl.load(file)

    return data




# Clean Train Trials

In [38]:
filename = '../data/human/anonymized/clean_main_trials.pkl'
clean_data = import_cleaned_data(filename)


In [39]:
clean_data.head()

,Unnamed: 0,Unnamed: 0.1,cur_gameloop,datetime,is_leader,joint_action,layout,layout_name,next_state,reward,...,state,time_elapsed,time_left,is_wait,completed,run,workerid_num,reward_norm,cur_gameloop_total,reward_norm_total
0,0,14459,0.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXPXX', 'O 2O', 'X1 X', 'XDXSX']",cramped_room,"{'players': [{'position': [1, 2], 'orientation...",0.0,...,"{'players': [{'position': [1, 2], 'orientation...",0.154,180.0,True,True,run-1,0,0.0,1203,0.0
1,1,14460,1.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXPXX', 'O 2O', 'X1 X', 'XDXSX']",cramped_room,"{'players': [{'position': [1, 2], 'orientation...",0.0,...,"{'players': [{'position': [1, 2], 'orientation...",0.312,180.0,True,True,run-1,0,0.0,1203,0.0
2,2,14461,2.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXPXX', 'O 2O', 'X1 X', 'XDXSX']",cramped_room,"{'players': [{'position': [1, 2], 'orientation...",0.0,...,"{'players': [{'position': [1, 2], 'orientation...",0.461,180.0,True,True,run-1,0,0.0,1203,0.0
3,3,14462,3.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXPXX', 'O 2O', 'X1 X', 'XDXSX']",cramped_room,"{'players': [{'position': [1, 2], 'orientation...",0.0,...,"{'players': [{'position': [1, 2], 'orientation...",0.614,179.0,True,True,run-1,0,0.0,1203,0.0
4,4,14463,4.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXPXX', 'O 2O', 'X1 X', 'XDXSX']",cramped_room,"{'players': [{'position': [1, 2], 'orientation...",0.0,...,"{'players': [{'position': [1, 2], 'orientation...",1.043,179.0,True,True,run-1,0,0.0,1203,0.0


In [40]:
clean_data.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'cur_gameloop', 'datetime', 'is_leader',
       'joint_action', 'layout', 'layout_name', 'next_state', 'reward',
       'round_num', 'round_type', 'score', 'state', 'time_elapsed',
       'time_left', 'is_wait', 'completed', 'run', 'workerid_num',
       'reward_norm', 'cur_gameloop_total', 'reward_norm_total'],
      dtype='object')

In [41]:
clean_data['layout_name'].unique()

array(['cramped_room', 'asymmetric_advantages', 'coordination_ring',
       'random3', 'random0'], dtype=object)

In [42]:
fc_data = clean_data[clean_data['layout_name']=='random0']

In [43]:
fc_data['run'].unique()

array(['run-1', 'run-2'], dtype=object)

In [44]:
fc_data['workerid_num'].unique()

array([ 0,  1,  2,  3,  4,  6,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 22, 23])

In [45]:
fc_data.groupby(['run','workerid_num', 'reward_norm_total']).size().reset_index().rename(columns={0:'count'})



,run,workerid_num,reward_norm_total,count
0,run-1,0,0.0,1203
1,run-1,1,380.0,1204
2,run-1,2,480.0,1204
3,run-1,3,360.0,1204
4,run-1,4,280.0,1204
5,run-1,6,0.0,1204
6,run-1,9,0.0,1204
7,run-2,10,440.0,1204
8,run-2,11,400.0,1204
9,run-2,12,160.0,1204


In [46]:
fc_data[fc_data['workerid_num']==1]['state'].to_numpy()[0]

"{'players': [{'position': [3, 1], 'orientation': [0, -1]}, {'position': [1, 2], 'orientation': [0, -1]}], 'objects': {}, 'order_list': ['onion'], 'pot_explosion': False}"

In [47]:
filename = '../data/human/anonymized/clean_train_trials.pkl'
clean_data = import_cleaned_data(filename)
fc_data = clean_data[clean_data['layout_name']=='random0']
fc_data.groupby(['run','workerid_num', 'reward_norm_total']).size().reset_index().rename(columns={0:'count'})




,run,workerid_num,reward_norm_total,count
0,run-1,2,480.0,1204
1,run-1,4,280.0,1204
2,run-2,15,260.0,1199
3,run-2,17,300.0,1204
4,run-2,19,400.0,1204
5,run-2,22,300.0,1136


In [48]:
filename = '../data/human/anonymized/clean_test_trials.pkl'
clean_data = import_cleaned_data(filename)
fc_data = clean_data[clean_data['layout_name']=='random0']
fc_data.groupby(['run','workerid_num', 'reward_norm_total']).size().reset_index().rename(columns={0:'count'})




,run,workerid_num,reward_norm_total,count
0,run-1,1,380.0,1204
1,run-1,3,360.0,1204
2,run-2,10,440.0,1204
3,run-2,11,400.0,1204
4,run-2,12,160.0,1204
5,run-2,13,300.0,1204


In [50]:
states_for_worker_1 = fc_data[fc_data['workerid_num']==1]['state']

In [51]:
states_for_worker_1

10827    {'players': [{'position': [3, 1], 'orientation...
10828    {'players': [{'position': [3, 1], 'orientation...
10829    {'players': [{'position': [3, 1], 'orientation...
10830    {'players': [{'position': [3, 1], 'orientation...
10831    {'players': [{'position': [3, 1], 'orientation...
                               ...                        
12026    {'players': [{'position': [3, 1], 'orientation...
12027    {'players': [{'position': [3, 2], 'orientation...
12028    {'players': [{'position': [3, 2], 'orientation...
12029    {'players': [{'position': [3, 3], 'orientation...
12030    {'players': [{'position': [3, 3], 'orientation...
Name: state, Length: 1204, dtype: object

# Clean Human AI Trials

In [11]:
filename = '../data/human/anonymized/clean_human_AI_trials.pkl'
human_ai_trials_data = import_cleaned_data(filename)


In [12]:
human_ai_trials_data

,action,cur_gameloop,datetime,indexOf,joint_action,layout,layout_name,model_type,next_state,player_index,...,score,state,template,time_elapsed,time_left,viewTime,reward_norm,cur_gameloop_total,reward_norm_total,workerid_num
482,NaN,0.0,1.558577e+12,NaN,"[[0, 0], [1, 0]]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [1, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [1, 2], 'orientation...",NaN,2.062,58.0,NaN,0.0,393,0.0,11
483,NaN,1.0,1.558577e+12,NaN,"[[1, 0], INTERACT]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [1, 2], 'orientation...",NaN,2.089,58.0,NaN,0.0,393,0.0,11
484,NaN,2.0,1.558577e+12,NaN,"[[0, 0], [-1, 0]]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [2, 2], 'orientation...",NaN,2.121,58.0,NaN,0.0,393,0.0,11
485,NaN,3.0,1.558577e+12,NaN,"[[0, 0], [0, -1]]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [2, 2], 'orientation...",NaN,2.273,58.0,NaN,0.0,393,0.0,11
486,NaN,4.0,1.558577e+12,NaN,"[[0, 0], INTERACT]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [2, 2], 'orientation...",NaN,2.416,58.0,NaN,0.0,393,0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137365,NaN,398.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.048,0.0,NaN,0.0,403,0.0,55
137366,NaN,399.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.199,0.0,NaN,0.0,403,0.0,55
137367,NaN,400.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.341,0.0,NaN,0.0,403,0.0,55
137368,NaN,401.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.489,0.0,NaN,0.0,403,0.0,55
